# RabbitMQ Tutorial with Pika

See for more info:
For RabbitMQ
https://www.rabbitmq.com/tutorials/tutorial-one-python.html
For Pika
https://pika.readthedocs.org

In [16]:
import pika

Create a channel and connect to it

In [30]:
connection = pika.BlockingConnection(pika.ConnectionParameters(
               'localhost'))
channel = connection.channel()

In [18]:
channel.queue_declare(queue='hello')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=4', 'queue=hello'])>"])>

In [ ]:
channel.basic_publish(exchange='',
                      routing_key='hello',
                      body='Hello World!')
print(" [x] Sent 'Hello World!'")

Receive a message

In [20]:
channel.queue_declare(queue='hello') #just in case it was closed/deleted before

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=5', 'queue=hello'])>"])>

In [21]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

Now we can consume the queue with that callback function

In [22]:
channel.basic_consume(callback, queue='hello',no_ack=False) #ack --> keep message until finished --> acknowledgement

'ctag1.bd6a94787d3b42d190e64cfb0982249b'

In [ ]:
print(' [*] Waitinhg for messages. To exit press CTRL+C')
channel.start_consuming()
# hint, interrupt the kernel 

## Message durability

In [36]:
import time

connection = pika.BlockingConnection(pika.ConnectionParameters(
        host='localhost'))
channel = connection.channel()

In [37]:

channel.queue_declare(queue='task_queue', durable=True)

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=1', 'message_count=0', 'queue=task_queue'])>"])>

In [38]:
message = "Hello World!"
channel.basic_publish(exchange='',
                      routing_key="task_queue",
                      body=message,
                      properties=pika.BasicProperties(
                         delivery_mode = 2, # make message persistent
                      ))

True

In [34]:
print(" [x] Sent %r" % message)

 [x] Sent 'Hello World!'


In [39]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
    time.sleep(body.count(b'.'))
    print(" [x] Done")
    ch.basic_ack(delivery_tag = method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(callback,
                      queue='task_queue')

channel.start_consuming()

KeyboardInterrupt: 

In [5]:
connection.close()